<a href="https://colab.research.google.com/github/EjbejaranosAI/TFM_multimodal/blob/main/Multimodal_Baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install required libraries

In [1]:
!pip install h5py validators tqdm numpy requests argparse colorama

## Getting CMOSEI

In [2]:
import sys
import os
from typing import Optional


In [4]:
# path to the SDK folder
SDK_PATH: Optional[str] = None
# path to the folder where you want to store data
DATA_PATH: Optional[str] = '/content/CMU-MultimodalSDK/data/'
# path to a pretrained word embedding file
WORD_EMB_PATH: Optional[str] = None
# path to loaded word embedding matrix and corresponding word2id mapping
CACHE_PATH: Optional[str] = '/content/CMU-MultimodalSDK/data//embedding_and_mapping.pt'

In [7]:
!git clone https://github.com/CMU-MultiComp-Lab/CMU-MultimodalSDK.git
colab = 'google.colab' in sys.modules
if colab:
    # noinspection PyBroadException
    try:
        %tensorflow_version 2.x
    except Exception:
        pass

if colab:

    os.environ['PROJECT_DIR'] = project_dir = '/content/CMU-MultimodalSDK'
    replace = False
    if os.path.isdir(project_dir):
        choice = input("Project already exists in folder "+
              "{}\nDelete the files and pull again? Enter Y/(N):\n"
              .format(project_dir))
        if choice.lower()=='y':
            !rm -r $PROJECT_DIR
            print("Deleted folder {}".format(project_dir))
        else:
            replace = False
            print("Nothing was changed.")
    if replace:
        print("Project cloned to " + project_dir)
    os.chdir(project_dir)
    print("Done.")

fatal: destination path 'CMU-MultimodalSDK' already exists and is not an empty directory.
Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Project already exists in folder /content/CMU-MultimodalSDK
Delete the files and pull again? Enter Y/(N):
N
Nothing was changed.
Done.


In [4]:
import sys

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

# create folders for storing the data
if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

# download highlevel features, low-level (raw) data and labels for the dataset MOSI
# if the files are already present, instead of downloading it you just load it yourself.
# here we use CMU_MOSI dataset as example.

DATASET = md.cmu_mosi

try:
    md.mmdataset(DATASET.highlevel, DATA_PATH)
except RuntimeError:
    print("High-level features have been downloaded previously.")

try:
    md.mmdataset(DATASET.raw, DATA_PATH)
except RuntimeError:
    print("Raw data have been downloaded previously.")

try:
    md.mmdataset(DATASET.labels, DATA_PATH)
except RuntimeError:
    print("Labels have been downloaded previously.")

SDK path is not specified! Please specify first in constants/paths.py


/content/CMU-MultimodalSDK/mmsdk/mmdatasdk/computational_sequence/file_ops.py:53: SyntaxWarning: "is" with a literal. Did you mean "=="?
  metadataHandle.create_dataset(metadataKey,(1,),dtype=h5py.special_dtype(vlen=unicode) if sys.version_info.major is 2 else h5py.special_dtype(vlen=str))
/content/CMU-MultimodalSDK/mmsdk/mmdatasdk/computational_sequence/file_ops.py:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
  cast_operator=unicode if sys.version_info.major is 2 else str


[2023-08-14 10:38:10.204] | Status  | Downloading from http://immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedWordVectors.csd to /content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedWordVectors.csd...


[2023-08-14 10:38:26.934] | Success | Download complete!
[2023-08-14 10:38:26.935] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedWordVectors.csd ...
[2023-08-14 10:38:26.943] | Status  | Checking the integrity of the <glove_vectors> computational sequence ...
[2023-08-14 10:38:26.943] | Status  | Checking the format of the data in <glove_vectors> computational sequence ...


[2023-08-14 10:38:26.984] | Success | <glove_vectors> computational sequence data in correct format.
[2023-08-14 10:38:26.984] | Status  | Checking the format of the metadata in <glove_vectors> computational sequence ...
[2023-08-14 10:38:26.984] | Warning | <glove_vectors> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:38:27.458] | Status  | Downloading from http://immortal.multicomp.cs.cmu.edu/CMU-MOSI/visual/CMU_MOSI_Visual_Facet_41.csd to /content/CMU-MultimodalSDK/data/CMU_MOSI_Visual_Facet_41.csd...


...

[2023-08-14 10:46:58.945] | Success | <words> computational sequence data in correct format.
[2023-08-14 10:46:58.945] | Status  | Checking the format of the metadata in <words> computational sequence ...
[2023-08-14 10:46:58.945] | Warning | <words> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:46:59.414] | Status  | Downloading from http://immortal.multicomp.cs.cmu.edu/CMU-MOSI/language/CMU_MOSI_TimestampedPhones.csd to /content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedPhones.csd...


[2023-08-14 10:47:01.076] | Success | Download complete!
[2023-08-14 10:47:01.077] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedPhones.csd ...
[2023-08-14 10:47:01.088] | Status  | Checking the integrity of the <phoneme> computational sequence ...
[2023-08-14 10:47:01.088] | Status  | Checking the format of the data in <phoneme> computational sequence ...


[2023-08-14 10:47:01.131] | Success | <phoneme> computational sequence data in correct format.
[2023-08-14 10:47:01.131] | Status  | Checking the format of the metadata in <phoneme> computational sequence ...
[2023-08-14 10:47:01.131] | Warning | <phoneme> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:47:01.131] | Success | Dataset initialized successfully ... 
[2023-08-14 10:47:01.609] | Status  | Downloading from http://immortal.multicomp.cs.cmu.edu/CMU-MOSI/labels/CMU_MOSI_Opinion_Labels.csd to /content/CMU-MultimodalSDK/data/CMU_MOSI_Opinion_Labels.csd...


[2023-08-14 10:47:02.794] | Success | Download complete!
[2023-08-14 10:47:02.796] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_Opinion_Labels.csd ...
[2023-08-14 10:47:02.803] | Status  | Checking the integrity of the <Opinion Segment Labels> computational sequence ...
[2023-08-14 10:47:02.803] | Status  | Checking the format of the data in <Opinion Segment Labels> computational sequence ...


[2023-08-14 10:47:02.846] | Success | <Opinion Segment Labels> computational sequence data in correct format.
[2023-08-14 10:47:02.846] | Status  | Checking the format of the metadata in <Opinion Segment Labels> computational sequence ...
[2023-08-14 10:47:02.846] | Warning | <Opinion Segment Labels> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:47:02.846] | Success | Dataset initialized successfully ... 


In [5]:
data_files = os.listdir(DATA_PATH)
print(len(data_files))
print('\n'.join(data_files))

11
CMU_MOSI_TimestampedWordVectors.csd
CMU_MOSI_Visual_OpenFace_1.csd
CMU_MOSI_openSMILE_IS09.csd
CMU_MOSI_TimestampedPhones.csd
CMU_MOSI_COVAREP.csd
CMU_MOSI_TimestampedWords.csd
CMU_MOSI_Opinion_Labels.csd
CMU_MOSI_Visual_Facet_42.csd
CMU_MOSI_OpenSmile_EB10.csd
CMU_MOSI_Visual_OpenFace_2.csd
CMU_MOSI_Visual_Facet_41.csd


In [10]:
import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError
# define your different modalities - refer to the filenames of the CSD files
visual_field = 'CMU_MOSI_Visual_Facet_41'
acoustic_field = 'CMU_MOSI_COVAREP'
text_field = 'CMU_MOSI_TimestampedWords'

features = [
    text_field,
    visual_field,
    acoustic_field
]

recipe = {feat: os.path.join(DATA_PATH, feat) + '.csd' for feat in features}
print(recipe)
dataset = md.mmdataset(recipe)

{'CMU_MOSI_TimestampedWords': '/content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedWords.csd', 'CMU_MOSI_Visual_Facet_41': '/content/CMU-MultimodalSDK/data/CMU_MOSI_Visual_Facet_41.csd', 'CMU_MOSI_COVAREP': '/content/CMU-MultimodalSDK/data/CMU_MOSI_COVAREP.csd'}
[2023-08-14 10:51:30.546] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_TimestampedWords.csd ...
[2023-08-14 10:51:30.559] | Status  | Checking the integrity of the <words> computational sequence ...
[2023-08-14 10:51:30.559] | Status  | Checking the format of the data in <words> computational sequence ...


[2023-08-14 10:51:30.619] | Success | <words> computational sequence data in correct format.
[2023-08-14 10:51:30.619] | Status  | Checking the format of the metadata in <words> computational sequence ...
[2023-08-14 10:51:30.620] | Warning | <words> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:51:30.621] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_Visual_Facet_41.csd ...
[2023-08-14 10:51:30.630] | Status  | Checking the integrity of the <FACET_4.1> computational sequence ...
[2023-08-14 10:51:30.631] | Status  | Checking the format of the data in <FACET_4.1> computational sequence ...


[2023-08-14 10:51:30.694] | Success | <FACET_4.1> computational sequence data in correct format.
[2023-08-14 10:51:30.696] | Status  | Checking the format of the metadata in <FACET_4.1> computational sequence ...
[2023-08-14 10:51:30.699] | Warning | <FACET_4.1> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:51:30.700] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_COVAREP.csd ...
[2023-08-14 10:51:30.709] | Status  | Checking the integrity of the <COVAREP> computational sequence ...
[2023-08-14 10:51:30.709] | Status  | Checking the format of the data in <COVAREP> computational sequence ...


[2023-08-14 10:51:30.779] | Success | <COVAREP> computational sequence data in correct format.
[2023-08-14 10:51:30.780] | Status  | Checking the format of the metadata in <COVAREP> computational sequence ...
[2023-08-14 10:51:30.780] | Warning | <COVAREP> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:51:30.781] | Success | Dataset initialized successfully ... 


In [11]:
print(list(dataset.keys()))
print("=" * 80)

print(list(dataset[visual_field].keys())[:10])
print("=" * 80)

some_id = list(dataset[visual_field].keys())[15]
print(list(dataset[visual_field][some_id].keys()))
print("=" * 80)

print(list(dataset[visual_field][some_id]['intervals'].shape))
print("=" * 80)

print(list(dataset[visual_field][some_id]['features'].shape))
print(list(dataset[text_field][some_id]['features'].shape))
print(list(dataset[acoustic_field][some_id]['features'].shape))
print("Different modalities have different number of time steps!")

['CMU_MOSI_TimestampedWords', 'CMU_MOSI_Visual_Facet_41', 'CMU_MOSI_COVAREP']
['03bSnISJMiM', '0h-zjBukYpk', '1DmNV9C1hbY', '1iG0909rllw', '2WGyTLYerpo', '2iD-tVS8NPw', '5W7Z1C_fDaE', '6Egk_28TtTM', '6_0THN4chvY', '73jzhE8R1TQ']
['features', 'intervals']
[5404, 2]
[5404, 47]
[645, 1]
[18009, 74]
Different modalities have different number of time steps!


In [12]:
# we define a simple averaging function that does not depend on intervals
def avg(intervals: np.array, features: np.array) -> np.array:
    try:
        return np.average(features, axis=0)
    except:
        return features

# first we align to words with averaging, collapse_function receives a list of functions
dataset.align(text_field, collapse_functions=[avg])

[2023-08-14 10:53:45.765] | Status  | Unify was called ...
[2023-08-14 10:53:45.766] | Success | Unify completed ...
[2023-08-14 10:53:45.766] | Status  | Pre-alignment based on <CMU_MOSI_TimestampedWords> computational sequence started ...
[2023-08-14 10:53:49.864] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2023-08-14 10:53:50.795] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2023-08-14 10:53:50.812] | Status  | Alignment starting ...


Aligning zhpQhgha_KU:  96%|█████████▋| 651/676 [00:01<00:00, 401.06 Segments/s]


[2023-08-14 10:56:00.852] | Success | Alignment to <CMU_MOSI_TimestampedWords> complete.
[2023-08-14 10:56:00.852] | Status  | Replacing dataset content with aligned computational sequences
[2023-08-14 10:56:00.860] | Success | Initialized empty <CMU_MOSI_TimestampedWords> computational sequence.
[2023-08-14 10:56:00.860] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWords> computational sequence ...


[2023-08-14 10:56:00.986] | Success | <CMU_MOSI_TimestampedWords> computational sequence data in correct format.
[2023-08-14 10:56:00.987] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWords> computational sequence ...
[2023-08-14 10:56:00.988] | Warning | <CMU_MOSI_TimestampedWords> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:56:00.988] | Success | Initialized empty <CMU_MOSI_Visual_Facet_41> computational sequence.
[2023-08-14 10:56:00.988] | Status  | Checking the format of the data in <CMU_MOSI_Visual_Facet_41> computational sequence ...


[2023-08-14 10:56:01.090] | Success | <CMU_MOSI_Visual_Facet_41> computational sequence data in correct format.
[2023-08-14 10:56:01.091] | Status  | Checking the format of the metadata in <CMU_MOSI_Visual_Facet_41> computational sequence ...
[2023-08-14 10:56:01.091] | Warning | <CMU_MOSI_Visual_Facet_41> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:56:01.092] | Success | Initialized empty <CMU_MOSI_COVAREP> computational sequence.
[2023-08-14 10:56:01.092] | Status  | Checking the format of the data in <CMU_MOSI_COVAREP> computational sequence ...


[2023-08-14 10:56:01.190] | Success | <CMU_MOSI_COVAREP> computational sequence data in correct format.
[2023-08-14 10:56:01.190] | Status  | Checking the format of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
[2023-08-14 10:56:01.190] | Warning | <CMU_MOSI_COVAREP> computational sequence does not have all the required metadata ... continuing 


In [13]:
label_field = 'CMU_MOSI_Opinion_Labels'

# we add and align to lables to obtain labeled segments
# this time we don't apply collapse functions so that the temporal sequences are preserved
label_recipe = {label_field: os.path.join(DATA_PATH, label_field + '.csd')}
dataset.add_computational_sequences(label_recipe, destination=None)
dataset.align(label_field)

[2023-08-14 10:58:06.360] | Success | Computational sequence read from file /content/CMU-MultimodalSDK/data/CMU_MOSI_Opinion_Labels.csd ...
[2023-08-14 10:58:06.370] | Status  | Checking the integrity of the <Opinion Segment Labels> computational sequence ...
[2023-08-14 10:58:06.370] | Status  | Checking the format of the data in <Opinion Segment Labels> computational sequence ...


[2023-08-14 10:58:06.411] | Success | <Opinion Segment Labels> computational sequence data in correct format.
[2023-08-14 10:58:06.412] | Status  | Checking the format of the metadata in <Opinion Segment Labels> computational sequence ...
[2023-08-14 10:58:06.412] | Warning | <Opinion Segment Labels> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:58:06.412] | Status  | Unify was called ...
[2023-08-14 10:58:06.492] | Success | Unify completed ...
[2023-08-14 10:58:06.495] | Status  | Pre-alignment based on <CMU_MOSI_Opinion_Labels> computational sequence started ...


[2023-08-14 10:58:06.789] | Status  | Pre-alignment done for <CMU_MOSI_COVAREP> ...
[2023-08-14 10:58:07.028] | Status  | Pre-alignment done for <CMU_MOSI_Visual_Facet_41> ...
[2023-08-14 10:58:07.174] | Status  | Pre-alignment done for <CMU_MOSI_TimestampedWords> ...
[2023-08-14 10:58:07.179] | Status  | Alignment starting ...


Aligning 03bSnISJMiM:   0%|          | 0/13 [00:00<?, ? Segments/s]
                                                                   
Aligning 0h-zjBukYpk:   0%|          | 0/25 [00:00<?, ? Segments/s]
                                                                   
Aligning 1iG0909rllw:   0%|          | 0/30 [00:00<?, ? Segments/s]
                                                                   
Aligning 2WGyTLYerpo:   0%|          | 0/63 [00:00<?, ? Segments/s]
                                                                   
Aligning 5W7Z1C_fDaE:   0%|          | 0/24 [00:00<?, ? Segments/s]
                                                                   
Aligning 6Egk_28TtTM:   0%|          | 0/12 [00:00<?, ? Segments/s]
                                                                   
Aligning 73jzhE8R1TQ:   0%|          | 0/19 [00:00<?, ? Segments/s]
                                                                   
Aligning 7JsX8y1ysxY:   0%|          | 0/39 [00:

[2023-08-14 10:58:12.292] | Success | Alignment to <CMU_MOSI_Opinion_Labels> complete.
[2023-08-14 10:58:12.292] | Status  | Replacing dataset content with aligned computational sequences
[2023-08-14 10:58:12.529] | Success | Initialized empty <CMU_MOSI_TimestampedWords> computational sequence.
[2023-08-14 10:58:12.531] | Status  | Checking the format of the data in <CMU_MOSI_TimestampedWords> computational sequence ...


[2023-08-14 10:58:12.544] | Success | <CMU_MOSI_TimestampedWords> computational sequence data in correct format.
[2023-08-14 10:58:12.545] | Status  | Checking the format of the metadata in <CMU_MOSI_TimestampedWords> computational sequence ...
[2023-08-14 10:58:12.546] | Warning | <CMU_MOSI_TimestampedWords> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:58:12.546] | Success | Initialized empty <CMU_MOSI_Visual_Facet_41> computational sequence.
[2023-08-14 10:58:12.546] | Status  | Checking the format of the data in <CMU_MOSI_Visual_Facet_41> computational sequence ...


[2023-08-14 10:58:12.559] | Success | <CMU_MOSI_Visual_Facet_41> computational sequence data in correct format.
[2023-08-14 10:58:12.560] | Status  | Checking the format of the metadata in <CMU_MOSI_Visual_Facet_41> computational sequence ...
[2023-08-14 10:58:12.560] | Warning | <CMU_MOSI_Visual_Facet_41> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:58:12.561] | Success | Initialized empty <CMU_MOSI_COVAREP> computational sequence.
[2023-08-14 10:58:12.561] | Status  | Checking the format of the data in <CMU_MOSI_COVAREP> computational sequence ...


[2023-08-14 10:58:12.574] | Success | <CMU_MOSI_COVAREP> computational sequence data in correct format.
[2023-08-14 10:58:12.574] | Status  | Checking the format of the metadata in <CMU_MOSI_COVAREP> computational sequence ...
[2023-08-14 10:58:12.575] | Warning | <CMU_MOSI_COVAREP> computational sequence does not have all the required metadata ... continuing 
[2023-08-14 10:58:12.575] | Success | Initialized empty <CMU_MOSI_Opinion_Labels> computational sequence.
[2023-08-14 10:58:12.576] | Status  | Checking the format of the data in <CMU_MOSI_Opinion_Labels> computational sequence ...


[2023-08-14 10:58:12.589] | Success | <CMU_MOSI_Opinion_Labels> computational sequence data in correct format.
[2023-08-14 10:58:12.589] | Status  | Checking the format of the metadata in <CMU_MOSI_Opinion_Labels> computational sequence ...
[2023-08-14 10:58:12.590] | Warning | <CMU_MOSI_Opinion_Labels> computational sequence does not have all the required metadata ... continuing 


In [14]:
# check out what the keys look like now
print(list(dataset[text_field].keys())[55])

1iG0909rllw[3]


## Splitting dataset

In [16]:
DATASET = md.cmu_mosi
# obtain the train/dev/test splits - these splits are based on video IDs
train_split = DATASET.standard_folds.standard_train_fold
dev_split = DATASET.standard_folds.standard_valid_fold
test_split = DATASET.standard_folds.standard_test_fold

# inspect the splits: they only contain video IDs
print(test_split)

['tmZoasNr4rU', 'zhpQhgha_KU', 'lXPQBPVc5Cw', 'iiK8YX8oH1E', 'tStelxIAHjw', 'nzpVDcQ0ywM', 'etzxEpPuc6I', 'cW1FSBF59ik', 'd6hH302o4v8', 'k5Y_838nuGo', 'pLTX3ipuDJI', 'jUzDDGyPkXU', 'f_pcplsH_V0', 'yvsjCA6Y5Fc', 'nbWiPyCm4g0', 'rnaNMUZpvvg', 'wMbj6ajWbic', 'cM3Yna7AavY', 'yDtzw_Y-7RU', 'vyB00TXsimI', 'dq3Nf_lMPnE', 'phBUpBr1hSo', 'd3_k5Xpfmik', 'v0zCBqDeKcE', 'tIrG4oNLFzE', 'fvVhgmXxadc', 'ob23OKe5a9Q', 'cXypl4FnoZo', 'vvZ4IcEtiZc', 'f9O3YtZ2VfI', 'c7UH_rxdZv4']


In [19]:
print(len(train_split))
print(len(dev_split))
print(len(test_split))

52
10
31


In [22]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm_notebook
from collections import defaultdict

EPS = 0

word2id = defaultdict(lambda: len(word2id))
UNK = word2id['<unk>']
PAD = word2id['<pad>']

train = []
dev = []
test = []

pattern = re.compile('(.*)\[.*\]')
num_drop = 0

for segment in dataset[label_field].keys():
  vid = re.search(pattern, segment).group(1)
  label = dataset[label_field][segment]['features']
  _words = dataset[text_field][segment]['features']
  _visual = dataset[visual_field][segment]['features']
  _acoustic = dataset[acoustic_field][segment]['features']

  # if the sequences are not same length after alignment, there must be some problem with some modalities
  # we should drop it or inspect the data again
  if not _words.shape[0] == _visual.shape[0] == _acoustic.shape[0]:
    print(f"Encountered datapoint {vid} with text shape {_words.shape}, visual shape {_visual.shape}, acoustic shape {_acoustic.shape}")
    num_drop += 1
    continue


  # remove nan values
  label = np.nan_to_num(label)
  _visual = np.nan_to_num(_visual)
  _acoustic = np.nan_to_num(_acoustic)

  #remove speech pause tokens - this is in general helpful
  # we should remove speech pauses and corresponding visual/acoustic features together
  # otherwise modalities whould no longer be aligned
  words = []
  visual = []
  acoustic = []
  for i, word in enumerate(_words):
    if word[0] != b'sp':
      words.append(word2id[word[0].decode('utf-8')])
      visual.append(_visual[i,:])
      acoustic.append(_acoustic[i,:])

  words = np.asarray(words)
  visual = np.asarray(visual)
  acoustic = np.asarray(acoustic)


  # Z-Normalization per instance and remove nan/infs
  visual = np.nan_to_num(visual-visual.mean(0,keepdims=True))/ (EPS + np.std(visual, axis=0, keepdims=True))
  acoustic = np.nan_to_num(acoustic - acoustic.mean(0,keepdims=True)) / (EPS + np.std(acoustic, axis=0, keepdims= True))

  if vid in train_split:
          train.append(((words, visual, acoustic), label, segment))
  elif vid in dev_split:
      dev.append(((words, visual, acoustic), label, segment))
  elif vid in test_split:
      test.append(((words, visual, acoustic), label, segment))
  else:
      print(f"Found video that doesn't belong to any splits: {vid}")

print(f"Total number of {num_drop} datapoints have been dropped.")
# Turn off the word2id - Define a named function here to allow for pickling
def return_unk():
  return UNK
word2id.default_factory = return_unk

<ipython-input-22-4d40fd2a5382>:60: RuntimeWarning: invalid value encountered in divide
  acoustic = np.nan_to_num(acoustic - acoustic.mean(0,keepdims=True)) / (EPS + np.std(acoustic, axis=0, keepdims= True))
<ipython-input-22-4d40fd2a5382>:59: RuntimeWarning: invalid value encountered in divide
  visual = np.nan_to_num(visual-visual.mean(0,keepdims=True))/ (EPS + np.std(visual, axis=0, keepdims=True))


Total number of 0 datapoints have been dropped.


/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:180: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:213: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:236: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


### Exploring dataset

In [28]:
print(len(train))
print(len(train))
print(len(train))

print(train[0][0][0].shape)
print(train[0][0][1].shape)
print(train[0][1])

print(f"Total vocab size: {len(word2id)}")

1283
1283
1283
(5,)
(5, 47)
[[2.4]]
Total vocab size: 2729


## Collate function

https://plainenglish.io/blog/understanding-collate-fn-in-pytorch-f9d1742647d3

Class to process and organize batches of data for training or evaluation. The function sorts the batch in descending order of length, extracts the data from the batch, pads the sequences using PyTorch's utility functions, and returns the padded sequences, labels, lengths, and other features of the batch


In [31]:
def multi_collate(batch):
  batch = sorted(batch, key=lambda x:x[0][0].shape[0], reverse=True)
  # Get the data out of the batch - Use pad sequence util functions from PyTorch to pad things
  labels = torch.cat([torch.from_numpy(sample[1]) for sample in batch], dim=0)
  sentences = pad_sequence([torch.LongTensor(sample[0][1]) for sample in batch])
  visual = pad_sequence([torch.FloatTensor(sample[0][1]) for sample in batch])
  acoustic = pad_sequence([torch.FloatTensor(sample[0][2]) for sample in batch])

  # Lengths are useful later in using RNNs
  lengths = torch.LongTensor([sample[0][0].shape[0] for sample in batch])
  return sentences, visual, acoustic, labels, lengths

# Construct dataloaders, dev and test could use around -x3 times batch size since no_grad is used
batch_size = 56
train_loader = DataLoader(train, shuffle=True, batch_size=batch_size, collate_fn=multi_collate)
dev_loader = DataLoader(dev, shuffle=True, batch_size=batch_size * 3, collate_fn=multi_collate)
test_loader = DataLoader(test, shuffle=True, batch_size=batch_size * 3, collate_fn=multi_collate)

# Create a temporary dataloader just to see how the batch looks like
temp_loader = iter(DataLoader(test, shuffle=True, batch_size=8, collate_fn=multi_collate))
batch = next(temp_loader)

print(f"# Word vectors, padded to maxlen : {batch[0].shape}")
print(f"# Visual vectors, padded to maxlen: {batch[1].shape}")
print(f"# Acoustic vectors, padded to maxlen: {batch[2].shape}")
print(f"# Labels: {batch[3]}")
print(f"# Lengths {batch[4]}")

# Word vectors, padded to maxlen : torch.Size([33, 8, 47])
# Visual vectors, padded to maxlen: torch.Size([33, 8, 47])
# Acoustic vectors, padded to maxlen: torch.Size([33, 8, 74])
# Labels: tensor([[-1.6000],
        [ 0.2000],
        [-1.6000],
        [ 2.0000],
        [ 2.8000],
        [ 2.0000],
        [-0.4000],
        [-0.6000]])
# Lengths tensor([33, 20, 15, 12, 11,  7,  6,  6])
